# This is a tutorial on how to use IOAI's provided LLM proxy

## Step 1: initialize your OpenAI client

Please use your provided api key here.

In [ ]:
from openai import AsyncClient

BASE_URL = "https://ioai-llm-proxy.up.railway.app/prox/v1"
API_KEY = "<YOUR_IOAI_API_KEY>"

openai_client = AsyncClient(
    base_url=BASE_URL,
    api_key=API_KEY,
)

## Step 2: Generate output

Note that we have an allowlist on model ids:

- `openai/gpt-4.1`
- `openai/gpt-4.1-mini`
- `openai/gpt-4.1-nano`
- `openai/gpt-4o`
- `openai/gpt-4o-mini`
- `gpt-4o-mini`
- `gpt-4o`
- `gpt-4.1`
- `gpt-4.1-mini`
- `gpt-4.1-nano`
- `google/gemini-2.5-pro`
- `google/gemini-2.5-flash`
- `moonshotai/kimi-k2`
- `qwen/qwen3-235b-a22b-07-25`
- `anthropic/claude-sonnet-4`

In [ ]:
response = await openai_client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, world!"}],
)

print(response)
print(response.choices[0].message.content)

## Advanced: Structured Outputs

You can use structured outputs to generate objects that fit a specific structure.

In [ ]:
from pydantic import BaseModel
from rich import print as rprint

class CalendarItem(BaseModel):
    title: str
    month: int
    date: int
    year: int
    description: str

result = await openai_client.beta.chat.completions.parse(
    model="openai/gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that generates calendar items."},
        {"role": "user", "content": "IOAI opening ceremony at Aug. 1, 2025"}
    ],
    response_format=CalendarItem,
)

print(result)
rprint(result.choices[0].message.parsed)

## Batch requests with retry

To generate large amounts of LLM completions efficiently and rhobustly, we can leverage:

- Concurrent requests capped by an async Semaphore -- so that we can make multiple requests at the same time without overwhelming our bandwidth
- Exponential backoff based retry for each request -- so that we gracefully retry when unexpected network / provider errors happen

The below is an example:

In [ ]:
class RankingExtraction(BaseModel):
    ranking: int
    contest: str

texts = [f"We ranked {i}th in the IOAI 2025" for i in range(1, 101)]
len(texts)

In [ ]:
from typing import List
from tenacity import retry, stop_after_attempt, wait_exponential
from asyncio import Semaphore
from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm as tqdm_asyncio

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=10),
)
async def process_one(txt: str) -> RankingExtraction:
    result = await openai_client.beta.chat.completions.parse(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": txt}],
        response_format=RankingExtraction,
    )
    return result.choices[0].message.parsed

In [ ]:
print(texts[0])
print(await process_one(texts[0]))

In [ ]:
async def process_all(texts: List[str]):
    semaphore = Semaphore(50) # we limit to making 50 requests concurrently
    async def _process_with_sema(txt: str):
        async with semaphore:
            return await process_one(txt)
    return await tqdm_asyncio.gather(
        *[_process_with_sema(txt) for txt in texts],
        desc="Processing",
        total=len(texts),
    )

results = await process_all(texts)

results[:10]

# Check your credits

You get $10 of credits, so use it economically! You can run the following cell to check the amount of credits you have used.

In [ ]:
from httpx import get

result = get(f"https://ioai-llm-proxy.up.railway.app/credits/{API_KEY}")

resp = result.json()

print(f"""
Credits limit: ${resp['limit']}
Used: ${resp['usage']}
Credits remaining: ${resp['limit'] - resp['usage']}
""")